In [1]:
import pandas as pd
import geopy.distance as distance
from ast import literal_eval as make_tuple

#adding lat/long/coord/distance

centroid_latlong_df = pd.read_csv('csv/statistical-area-2-2018-centroid-inside.csv')

#read commuting to school data
df = pd.read_csv('csv/2018.csv')

#replace all suppresed values with 0
df.replace(-999, 0, inplace=True)

In [2]:
#add new total based only on published values
df['Total_without_suppresed_values'] = df['Study_at_home'] + df['Drive_a_car_truck_or_van'] + df['Passenger_in_a_car_truck_or_van'] + df['Train'] + df['Bicycle'] + df['Walk_or_jog'] + df['School_bus'] + df['Public_bus'] + df['Ferry'] + df['Other']

In [3]:
#turn first POINT(LONG, LAT) col into tuples
centroid_latlong_df['WKT'] = centroid_latlong_df['WKT'].apply(lambda x: make_tuple(x[6:].replace(' ', ',')))
#switch (long, lat) into (lat, long) to match the distance function args
centroid_latlong_df['WKT'] = centroid_latlong_df['WKT'].apply(lambda x: (x[1], x[0]))
centroid_latlong_df['WKT']

0        (-35.6294436685506, 174.28665672124)
1       (-44.0970108698041, 170.825353451148)
2       (-34.5054527643316, 172.775549967099)
3       (-34.9162774394026, 173.137442823842)
4       (-35.2185013354381, 174.158249405953)
                        ...                  
2232      (-43.9582203749606, 172.7389225487)
2233    (-46.1322836159479, 170.343623564645)
2234    (-37.8523753096606, 175.293653883253)
2235    (-43.0689157697391, 171.672755166319)
2236    (-43.3456883294022, 171.903203084593)
Name: WKT, Length: 2237, dtype: object

In [4]:
def add_lat_long_point(place):
    lat = place + '_lat'
    long = place + '_long'
    point = place + '_point'
    for col in [lat, long, point]:
        df[col] = ''
    for i, c in enumerate(df[place + '_' + 'code']):
        for j, code in enumerate(centroid_latlong_df['code']):
            if c == code:
                df[lat][i] = centroid_latlong_df['latitude'][j] 
                df[long][i] = centroid_latlong_df['longitude'][j] 
                df[point][i] = centroid_latlong_df['WKT'][j]

In [5]:
#This may take a while!

add_lat_long_point('address')
add_lat_long_point('school')

In [6]:
#check to make sure all coords are added
df[['address', 'address_lat', 'address_long', 'address_point', 'school_address', 'school_lat', 'school_long', 'school_point']]

,address,address_lat,address_long,address_point,school_address,school_lat,school_long,school_point
0,North Cape,-34.5055,172.776,"(-34.5054527643316, 172.775549967099)",North Cape,-34.5055,172.776,"(-34.5054527643316, 172.775549967099)"
1,Rangaunu Harbour,-34.9163,173.137,"(-34.9162774394026, 173.137442823842)",Rangaunu Harbour,-34.9163,173.137,"(-34.9162774394026, 173.137442823842)"
2,Karikari Peninsula,-34.9953,173.379,"(-34.995278160775, 173.378738365012)",Karikari Peninsula,-34.9953,173.379,"(-34.995278160775, 173.378738365012)"
3,Tangonge,-35.1231,173.219,"(-35.1231472715024, 173.218604066373)",Tangonge,-35.1231,173.219,"(-35.1231472715024, 173.218604066373)"
4,Ahipara,-35.1595,173.161,"(-35.1595169664155, 173.160919377723)",Ahipara,-35.1595,173.161,"(-35.1595169664155, 173.160919377723)"
...,...,...,...,...,...,...,...,...
22425,Owhata East,-38.1378,176.298,"(-38.1378419704118, 176.298143815984)",Lynmore,-38.1494,176.289,"(-38.1494057809273, 176.288953622991)"
22426,Aranui,-43.5106,172.705,"(-43.510603582444, 172.705323182291)",St Martins,-43.5572,172.654,"(-43.5572164003909, 172.653955597541)"
22427,Teviot Valley,-45.5823,169.427,"(-45.5822705917806, 169.426678965164)",Alexandra South,-45.2521,169.376,"(-45.2521042100642, 169.375785410127)"
22428,Woodbourne,-41.5308,173.904,"(-41.5308030990199, 173.903637938379)",Redwoodtown West,-41.53,173.949,"(-41.529955595641, 173.948580912091)"


In [7]:
#create a new col for distance:
df['distance'] = ''
#loop over distance col and calculate distance between address and school using distance() from geopy lib.
for i, d in enumerate(df['distance']):
    df['distance'][i] = distance.distance(df['address_point'][i], df['school_point'][i]).km

In [8]:
df['distance']

0               0
1               0
2               0
3               0
4               0
           ...   
22425     1.51545
22426      6.6378
22427     36.9103
22428     3.75199
22429    0.809982
Name: distance, Length: 22430, dtype: object

In [9]:
#number of cases where ppl travel within an area (distance = 0) => can calculate the percentage of those who study/work out of their area
len(df[df['distance'] == 0])

2059

In [10]:
#Double-check! The distance should be 0 when address and school adress are the same! So our calculations are correct
len(df[df['address'] == df['school_address']])

2059

In [11]:
#NOTE: adding region
region_df = pd.read_csv('./csv/geographic_regions.csv')

#replace region relevant col names for better readability
region_df.rename(columns = {'SA22018_name': 'statistical_area', 'UR2018_name': 'urban_rural_descriptor', 'IUR2018_name' : 'urban_rural_indicator_descriptor', 'TA2018_name' : 'territorial_authority', 'AU2017_name' : 'area_unit', 'UA2017_name' : 'urban_area'}, inplace=True)

In [12]:
#a function for looking up each region category by stat_area and adding the corresponding value to the main df
def add_region_category(category):
    #create a look-up df with 2 cols: stat_area and the region category in question
    lookup_df = region_df[['statistical_area', category]].drop_duplicates().reset_index()
    for i, address in enumerate(df['address']):
        for j, area in enumerate(lookup_df['statistical_area']):
            if address == area:
                df[category][i] = lookup_df[category][j]
                break

In [13]:
#make a list of new region cols to be added to df. The names must match up with the names in the region_df
region_cols = ['urban_rural_descriptor', 'urban_rural_indicator_descriptor', 'territorial_authority', 'area_unit', 'urban_area']

#for each col, make a new col in df and apply the above function to fill the col with the matching regional category/label. This may take a while!
for col in region_cols:
    df[col] = ''
    add_region_category(col)

In [14]:
#check to make sure that all categories have been added to the right address
region_cols.append('address')
df[region_cols]

,urban_rural_descriptor,urban_rural_indicator_descriptor,territorial_authority,area_unit,urban_area,address
0,Other rural Far North District,Rural other,Far North District,North Cape,Rural (Incl.some Off Shore Islands),North Cape
1,Other rural Far North District,Rural other,Far North District,Motutangi-Kareponia,Rural (Incl.some Off Shore Islands),Rangaunu Harbour
2,Other rural Far North District,Rural other,Far North District,Karikari Peninsula-Maungataniwha,Rural (Incl.some Off Shore Islands),Karikari Peninsula
3,Other rural Far North District,Rural other,Far North District,Motutangi-Kareponia,Rural (Incl.some Off Shore Islands),Tangonge
4,Ahipara,Rural settlement,Far North District,Ahipara,Rural Centre,Ahipara
...,...,...,...,...,...,...
22425,Rotorua,Large urban area,Rotorua District,Owhata East,Rotorua,Owhata East
22426,Christchurch,Major urban area,Christchurch City,Aranui,Christchurch,Aranui
22427,Other rural Central Otago District,Rural other,Central Otago District,Teviot,Rural (Incl.some Off Shore Islands),Teviot Valley
22428,Other rural Marlborough District,Rural other,Marlborough District,Omaka,Blenheim,Woodbourne


In [15]:
#the number of distinct labels in each categorization scheme:
for scheme in ['urban_rural_descriptor', 'urban_rural_indicator_descriptor', 'territorial_authority', 'area_unit', 'urban_area']:
    print('Number of Unique ' + scheme.replace('_', ' ') + ': ' + str(len(df[scheme].unique())))

#the order of columns will be ascending (general=>specific) (in case we want to make a sunburst chart). Still, these schemes should be used independently (???)

Number of Unique urban rural descriptor: 317
Number of Unique urban rural indicator descriptor: 7
Number of Unique territorial authority: 67
Number of Unique area unit: 1390
Number of Unique urban area: 141


In [16]:
#reorder the columns in df before saving to csv (address/stat area should be the most specific) (remove easting/northing)
clean_df = df[['urban_rural_indicator_descriptor', 'territorial_authority', 'urban_area', 'urban_rural_descriptor', 'area_unit', 'address', 'address_code', 'address_lat', 'address_long', 'address_point', 'school_address', 'school_code', 'school_lat', 'school_long', 'school_point', 'distance', 'Study_at_home', 'Drive_a_car_truck_or_van','Passenger_in_a_car_truck_or_van', 'Train', 'Bicycle', 'Walk_or_jog','School_bus', 'Public_bus', 'Ferry', 'Other', 'Total', 'Total_without_suppresed_values']]

pd.DataFrame.to_csv(clean_df, './csv/cleaned_df.csv')